# Neural machine translation (Aplicaciones de Procesamiento del Lenguaje Natural)

In this colab notebook you will learn how to work with a pre-trained model from [Huggingface](https://huggingface.co/) and how to fine tune such a model to improve its performace on your texts.

----
The code in this colab notebook is insprired in the content of the following websites:
* https://medium.com/@tskumar1320/how-to-fine-tune-pre-trained-language-translation-model-3e8a6aace9f
* https://huggingface.co/docs/transformers/training
* https://huggingface.co/docs/transformers/main_classes/tokenizer
* https://huggingface.co/docs/evaluate/


## Install the required libraries

In [ ]:
! pip install transformers[torch,sentencepiece]
! pip install datasets
! pip install evaluate
! pip install sacremoses
! pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

# Daniel Asensi 48776120C


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from google.colab import drive
import torch
import evaluate
from transformers import pipeline, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback, AutoTokenizer
from transformers import GenerationConfig, PreTrainedTokenizerBase, PreTrainedModel
import numpy as np
from functools import partial
from shutil import rmtree
import os
import torch
from tqdm.auto import tqdm


drive.mount("/content/drive", force_remount=True)
mydrive="/content/drive/MyDrive/apln/"

# Estos ficheros ya los he pasado por el script por lo que deberían estar limpios
# Descomenta estas lineas si quieres ver como he generado los datasets
# source_path = "/content/drive/MyDrive/apln/" + "fichero-tmp.en"
# target_path = "/content/drive/MyDrive/apln/" + "fichero-tmp.es"

path_to_save = "/content/drive/MyDrive/apln/"

# Aunque los modelos no van en cpu para hacer el desarrollo lo necesitaba
device = torch.device("cuda:0"if torch.cuda.is_available() else "cpu")
print(device)

# Elimina la carpeta de models donde guardaré los modelos
# posteriormente para poder cargarlos
if os.path.isdir('models'):
  print("Eliminando Models")
  rmtree("models")

Mounted at /content/drive
cuda:0


## Creacion de los Corpus



### Creación de los corpus a mano

Esta es la manera en la que he creado mis corpus personalizados a partir de un solo corpus, **este codigo no es necesario que lo ejecutes solo es necesaria la función load_text_file**

In [14]:
# Esto es para leerlos linea a linea por el tema del moses
def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.read().splitlines()
    return lines

In [ ]:
# Carga los archivos como listas de líneas
lines_en = load_text_file(source_path)
lines_es = load_text_file(target_path)

df_en = pd.DataFrame(lines_en, columns=['en'])
df_es = pd.DataFrame(lines_es, columns=['es'])

tam_bi = 5000 # 5000
tam_de = 2000 # 2000
tam_mon = 5000 # 5000

print(len(lines_en))
print(len(lines_es))
print(lines_es[0])
print(lines_en[0])
print(lines_es[400])
print(lines_en[400])
print(lines_es[-1])
print(lines_en[-1])
print("--------------------------------")
# Corpus bilingue de 5k lineas
primeras_5000_en = df_en.head(tam_bi)
primeras_5000_es = df_es.head(tam_bi)
primeras_5000_en.to_csv(path_to_save + 'bilingue.en', index=False, header=False)
primeras_5000_es.to_csv(path_to_save + 'bilingue.es', index=False, header=False)
df_en = df_en.iloc[tam_bi:].reset_index(drop=True)
df_es = df_es.iloc[tam_bi:].reset_index(drop=True)
print(df_en.iloc[0])
print(df_es.iloc[0])
print("--------------------------------")
# Corpus bilingue de 2k lineas Dev
primeras_5000_en = df_en.head(tam_de)
primeras_5000_es = df_es.head(tam_de)
primeras_5000_en.to_csv(path_to_save + 'Dev.en', index=False, header=False)
primeras_5000_es.to_csv(path_to_save + 'Dev.es', index=False, header=False)
df_en = df_en.iloc[tam_de:].reset_index(drop=True)
df_es = df_es.iloc[tam_de:].reset_index(drop=True)
print(df_en.iloc[0])
print(df_es.iloc[0])
print("--------------------------------")
# Corpus bilingue de 2k lineas Test
primeras_5000_en = df_en.head(tam_de)
primeras_5000_es = df_es.head(tam_de)
primeras_5000_en.to_csv(path_to_save + 'Test.en', index=False, header=False)
primeras_5000_es.to_csv(path_to_save + 'Test.es', index=False, header=False)
df_en = df_en.iloc[tam_de:].reset_index(drop=True)
df_es = df_es.iloc[tam_de:].reset_index(drop=True)
print(df_en.iloc[0])
print(df_es.iloc[0])
print("--------------------------------")
# Corpus Monolingues de 5k lineas cada uno
primeras_5000_en = df_en.head(tam_mon)
ultimas_5000_es = df_es.tail(tam_mon)
primeras_5000_en.to_csv(path_to_save + 'MonolingueEN.txt', index=False, header=False)
ultimas_5000_es.to_csv(path_to_save + 'MonolingueES.txt', index=False, header=False)

# Creación de los modelos

En este apartado de código defino los nombres de los modelos que voy a utilizar, así como los parámetros que más he modificado durante el desarrollo de la práctica, además aquí defino y los tokenizadores de cada uno de los modelos y sus data_collator

In [3]:
patience = 3 # 5
batch_size = 64
num_epochs = 3 # 30
lr = 2e-4 # 2e-5


model_A_name = "Helsinki-NLP/opus-mt-en-es"
model_B_name = "Helsinki-NLP/opus-mt-es-en"

output_model_name_A = "modelA-en-es"
output_model_name_B = "modelA-es-en"

model_A_path = "./models/EN-ES-MODEL"
model_B_path = "./models/ES-EN-MODEL"

tokenizer_A = AutoTokenizer.from_pretrained(model_A_name)
tokenizer_B = AutoTokenizer.from_pretrained(model_B_name)

data_collator_A = DataCollatorForSeq2Seq(tokenizer_A, model=model_A_name)
data_collator_B = DataCollatorForSeq2Seq(tokenizer_B, model=model_B_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

Aquí defini los argumentos del entrenamiento, estos se utilizarán más adelante en la función de entrenamiento.

In [4]:
argsTarget2Source = Seq2SeqTrainingArguments(
    output_dir = "./ouputModelTarget2Source",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = num_epochs,
    predict_with_generate = True,
    fp16 = True,
    metric_for_best_model = "bleu",
    load_best_model_at_end = True,
)

argsSource2Target = Seq2SeqTrainingArguments(
    output_dir = "./ouputModelSource2Target",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = num_epochs,
    predict_with_generate = True,
    fp16 = True,
    metric_for_best_model = "bleu",
    load_best_model_at_end = True,
  )

### Funciones auxiliares para la creación y tokenización de la entrada y salida de los modelos

In [5]:
def create_translation_dataset(source_path, target_path, source_lang='en', target_lang='es'):
    with open(source_path, 'r', encoding='utf-8') as source_file, open(target_path, 'r', encoding='utf-8') as target_file:
        source_data = source_file.readlines()
        target_data = target_file.readlines()

    dataset = Dataset.from_dict({
        'translation': [{source_lang: src.strip(), target_lang: tgt.strip()} for src, tgt in zip(source_data, target_data)]
    })

    return dataset

In [7]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples, source, target, tokenizer):
    inputs = [ex[source] for ex in examples["translation"]]
    targets = [ex[target] for ex in examples["translation"]]
    model_inputs = tokenizer(text=inputs, max_length=max_input_length, padding=True, truncation=True)
    labels = tokenizer(text=targets, max_length=max_target_length, padding=True, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

### Funciones para la evaluación de los 2 modelos

El por que de la realización de las 2 funciones  de evaluación en vez de una única es que no podía parar el tokenizer por parámetro y al ser diferente para los 2 modelos, pues tuve que repetir código

In [6]:
metric = evaluate.load("sacrebleu")

def compute_metrics_model_A(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer_A.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer_A.pad_token_id)
    decoded_labels = tokenizer_A.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": round(result["score"], 4)}

def compute_metrics_model_B(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer_B.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer_B.pad_token_id)
    decoded_labels = tokenizer_B.batch_decode(labels, skip_special_tokens=True)
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": round(result["score"], 4)}

### Funciones que compone el Iterative Back Translation

Es función se utiliza para combinar las traducciones de los textos monolingues con el corpus bilingue, pero sin concatenar

In [8]:
def combine_datasets(original_dataset, new_translations, target_texts, source_lang='en', target_lang='es'):
    """
    Combina el corpus bilingüe original con nuevas traducciones para crear un dataset aumentado.

    original_dataset: Lista de diccionarios que representan el dataset bilingüe original o un objeto Dataset.
    new_translations: Lista de nuevas traducciones para combinar con el original.
    target_texts: Lista de textos en el idioma objetivo que corresponden a las nuevas traducciones.
    source_lang: Código de idioma fuente en el par de traducción (por defecto 'en').
    target_lang: Código de idioma objetivo en el par de traducción (por defecto 'es').
    """

    # Convertir el Dataset de Hugging Face a una lista de diccionarios si es necesario
    if isinstance(original_dataset, Dataset):
        original_dataset = original_dataset.to_dict()['translation']

    # Asegurar que new_translations y target_texts tienen la misma longitud
    if len(new_translations) != len(target_texts):
        raise ValueError("La longitud de las nuevas traducciones y los textos objetivo debe ser la misma.")

    # Combinar las nuevas traducciones con el dataset original
    combined_data = original_dataset[:]
    for new_translation, target_text in zip(new_translations, target_texts):
        combined_data.append({
            source_lang: new_translation,
            target_lang: target_text
        })

    # Convertir la lista combinada de nuevo a un objeto Dataset
    combined_dataset = Dataset.from_dict({'translation': combined_data})

    return combined_dataset


Estas funciones son las utilizadas para generar las traducciones de los modelos sobre el texto monolingue

In [11]:
# Esta función creo que va más lenta
def generate_translations(model_name, texts, tokenizer):
    # Inicializar la pipeline de traducción
    translation_pipeline = pipeline("translation", model=model_name, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

    # Realizar la traducción usando los parámetros personalizados
    translations = translation_pipeline(texts)

    # Extraer el texto de las traducciones y devolverlo
    return [translation['translation_text'] for translation in translations]


# Esta función por lo que he probado es más rapida
def generate_translations(model, texts, tokenizer):
    """
    model: El modelo preentrenado de Hugging Face para la traducción.
    tokenizer: El tokenizer asociado al modelo.
    texts: Lista de textos en el idioma fuente para traducir.
    """

    model.to(device)
    model.eval()

    translations = []

    for text in tqdm(texts, desc="Translating"):
        # Codificar
        encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)

        # Generar la salida traducida
        with torch.no_grad():
            output_tokens = model.generate(**encoded_input)

        # Decodificar
        translation = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

        translations.append(translation)

    return translations

def update_dataset_with_translations(original_dataset, translations, target_lang, source_lang):
    updated_dataset = original_dataset.add_column(f"{source_lang}_translations", translations)
    return updated_dataset

Esta es la función de entrenamiento la cual recibe todos los parámetros que necesitamos, a destacar el último que es la letra del modelo, este los paso por lo que he comentado anteriormente del compute_metrics, esta función devuelve el modelo entrenado y además lo guarda en un carpeta destino para luego cargarlo y usarlo

In [12]:
def train_model(model_name, model_save_path, train_dataset, eval_dataset, training_args, data_collator, tokenizer, letter):
    """
    model_name: El nombre del modelo preentrenado o la ruta del modelo a cargar para continuar el entrenamiento.
    model_save_path: Ruta donde se guardará el modelo entrenado.
    train_dataset: El conjunto de datos de entrenamiento.
    eval_dataset: El conjunto de datos de evaluación.
    training_args: Los argumentos de entrenamiento (de la clase TrainingArguments).
    data_collator: El DataCollator utilizado para el entrenamiento y evaluación.
    tokenizer: El tokenizer asociado al modelo.
    letter: letra a la que hace referencia el modelo
    """
    # Verificar si ya existe un modelo entrenado y cargarlo, de lo contrario, cargar el modelo preentrenado
    if os.path.exists(model_save_path):
        print("Cargando modelo entrenado previamente desde:", model_save_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_save_path).to(training_args.device)
    else:
        print("Cargando modelo preentrenado:", model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(training_args.device)

    if letter == "B":
      trainer = Seq2SeqTrainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
          tokenizer=tokenizer,
          compute_metrics=compute_metrics_model_B,
          callbacks=[EarlyStoppingCallback(early_stopping_patience=patience)]
      )
    else:
      trainer = Seq2SeqTrainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
          tokenizer=tokenizer,
          compute_metrics=compute_metrics_model_A,
          callbacks=[EarlyStoppingCallback(early_stopping_patience=patience)]
      )

    trainer.train()

    # Guardar el modelo y el tokenizer entrenados
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

    return model

### Carga y tokenización de los datasets en memoria desde google drive

In [16]:
# Cargar el corpus bilingüe
corpus_bilingue_A_B = create_translation_dataset(
    source_path=path_to_save + 'bilingue.en',
    target_path=path_to_save + 'bilingue.es',
    source_lang='en',
    target_lang='es'
)

corpus_bilingue_B_A = create_translation_dataset(
    source_path=path_to_save + 'bilingue.es',
    target_path=path_to_save + 'bilingue.en',
    source_lang='es',
    target_lang='en'
)


# Cargar los corpus de desarrollo y prueba
corpus_dev_A_B = create_translation_dataset(
    source_path=path_to_save + 'Dev.en',
    target_path=path_to_save + 'Dev.es',
    source_lang='en',
    target_lang='es'
)

corpus_dev_B_A = create_translation_dataset(
    source_path=path_to_save + 'Dev.es',
    target_path=path_to_save + 'Dev.en',
    source_lang='es',
    target_lang='en'
)

print(corpus_dev_B_A)

tokenized_dev_A_B = corpus_dev_A_B.map(partial(preprocess_function, source="en", target="es", tokenizer=tokenizer_A), batched=True)
tokenized_dev_B_A = corpus_dev_B_A.map(partial(preprocess_function, source="es", target="en",  tokenizer=tokenizer_B), batched=True)

corpus_test_A_B = create_translation_dataset(
    source_path=path_to_save + 'Test.en',
    target_path=path_to_save + 'Test.es',
    source_lang='en',
    target_lang='es'
)

corpus_test_B_A = create_translation_dataset(
    source_path=path_to_save + 'Test.es',
    target_path=path_to_save + 'Test.en',
    source_lang='es',
    target_lang='en'
)

# Cargar los corpus monolingües (aunque no son directamente usados en la función como está escrita, podrían ser necesarios para la evaluación o futuras iteraciones)
corpus_monolingue_en = load_text_file(path_to_save + 'MonolingueEN.txt')
corpus_monolingue_es = load_text_file(path_to_save + 'MonolingueES.txt')
print(corpus_monolingue_es[0])
print(corpus_monolingue_en[0])

Dataset({
    features: ['translation'],
    num_rows: 2000
})


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

"A corto plazo, la RC3 moverá a los inversores a correr riesgo y estimulará una modesta reflación de los activos, pero es probable que el aumento del precio de los valores se disipe con el tiempo, si el crecimiento económico resulta decepcionante, como es probable, y hunde las esperanzas en materia de ingresos y rentabilidad empresariales."
Officials and others who favor stimulating growth through increased government spending ignore the CBO’s more realistic alternative scenario.


## Función de Iterative Back Translation

In [17]:
def iterativeBackTranslation():

    corpus_aumentado_B_A = corpus_bilingue_B_A

    for iteration in range(1, 4):

        print(f"Iteración {iteration}:")

        if iteration == 1:
          # Paso 1: Entrenar modelo B → A
          print("Entrenando modelo B → A... ES EN")
          tokenized_bilingue_B_A = corpus_bilingue_B_A.map(partial(preprocess_function, source="es", target="en", tokenizer=tokenizer_B), batched=True)
          model_B2A = train_model(model_B_name, model_B_path, tokenized_bilingue_B_A, tokenized_dev_B_A, argsTarget2Source, data_collator_B, tokenizer_B, "B")
        else:
          # Paso 1: Entrenar modelo B → A
          print("Entrenando modelo B → A... ES EN CORPUS AUMENTADO")
          tokenized_bilingue_B_A = corpus_aumentado_B_A.map(partial(preprocess_function, source="es", target="en", tokenizer=tokenizer_B), batched=True)
          model_B2A = train_model(model_B_name, model_B_path, tokenized_bilingue_B_A, tokenized_dev_B_A, argsTarget2Source, data_collator_B, tokenizer_B, "B")

        # Paso 2: Traducir corpus monolingüe B a A'
        print("Generando traducciones de B a A'... ES EN")
        translations_B2A = generate_translations(model_B2A, corpus_monolingue_es, tokenizer=tokenizer_B)

        # Paso 3: Combinar el corpus bilingüe original con A'-B
        print("Obteniendo A'...")
        combined_dataset_AB_A_primeB = combine_datasets(corpus_bilingue_B_A, translations_B2A, corpus_monolingue_es, source_lang='en', target_lang='es')

        # Paso 4: Entrenar modelo A → B con el nuevo corpus bilingüe aumentado
        print("Entrenando modelo A → B con el corpus aumentado...")
        tokenized_combined_dataset_AB_A_primeB = combined_dataset_AB_A_primeB.map(partial(preprocess_function, source="en", target="es", tokenizer=tokenizer_A), batched=True)
        model_A2B = train_model(model_A_name, model_A_path, tokenized_combined_dataset_AB_A_primeB, tokenized_dev_A_B, argsSource2Target, data_collator_A, tokenizer_A, "A")

        # Paso 5: Traducir corpus monolingüe A para obtener B'
        print("Generando traducciones de A a B'...")
        translations_A2B = generate_translations(model_A2B, corpus_monolingue_en, tokenizer=tokenizer_A)
        print(translations_A2B)

        # Paso 6: Combinar el corpus bilingüe original con A-B'
        combined_dataset_AB_prime = combine_datasets(corpus_bilingue_A_B, translations_A2B, corpus_monolingue_en, source_lang='es', target_lang='en')

        # Actualizar el corpus bilingüe original para la siguiente iteración
        corpus_aumentado_B_A = combined_dataset_AB_prime

        print(f"Finalizada iteración {iteration}.\n")
        print("----------------------------------------------------------------")

    return model_B2A, model_A2B



In [ ]:
model_B2A, model_A2B = iterativeBackTranslation()

Iteración 1:
Entrenando modelo B → A... ES EN


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Cargando modelo preentrenado: Helsinki-NLP/opus-mt-es-en


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,No log,0.474241,38.908800
2,No log,0.424653,40.903800
3,No log,0.417004,41.413100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

Generando traducciones de A a B'...


Translating:   0%|          | 0/5000 [00:00<?, ?it/s]

## Función de test

In [ ]:
def test_model(test_dataset, model_name, source_lang, target_lang, device='cuda:0', batch_size=64):
    """
    test_dataset: Un conjunto de datos que contiene pares de oraciones fuente y de referencia.
    model_name: Ruta al modelo preentrenado.
    source_lang: La abreviatura del idioma fuente
    target_lang: La abreviatura del idioma objetivo
    device: El dispositivo en el que se ejecutará la traducción
    batch_size: El tamaño del lote para la traducción.

    """
    # Inicializa el bleu
    metric = evaluate.load("sacrebleu")

    # Crea la pipeline de traducción
    translator = pipeline("translation", model=model_name, device=device, batch_size=batch_size)

    # Extrae las oraciones del conjunto de datos de prueba
    inputs = [ex[source_lang] for ex in test_dataset["translation"]]
    references = [[ex[target_lang]] for ex in test_dataset["translation"]]

    # Genera traducciones
    pre_outputs = translator(inputs)
    outputs = [ex["translation_text"] for ex in pre_outputs]

    # Calcula la puntuación bleu
    result = metric.compute(predictions=outputs, references=references)

    del translator

    return result


In [ ]:
result = test_model(test_dataset=corpus_test_A_B, model_name=model_A2B, source_lang='en', target_lang='es', device=device, batch_size=batch_size)
print(result)

result = test_model(test_dataset=corpus_test_B_A, model_name=model_B2A, source_lang='es', target_lang='en', device=device, batch_size=batch_size)
print(result)